In [ ]:
from pathlib import Path

import pandas as pd

import seaborn as sns

sns.set_style("darkgrid")

In [ ]:
%pwd

In [ ]:
# We have to move up to the parent directory so that we can import things from utils
%cd ..
%pwd


In [ ]:
import utils

In [ ]:
P_ROOT = Path(".").resolve()
print(f"project root: {P_ROOT}")

In [ ]:
redcap_fname = Path("/Volumes/Macintosh HD/Users/sealab/MRI_Processing/BABIES-Tracking/csv/redcap_BABIES.csv")
redcap_datadict_fname = Path("/Volumes/Macintosh HD/Users/sealab/MRI_Processing/BABIES-Tracking/csv/BABIES_DataDictionary.csv")
assert redcap_fname.exists()
assert redcap_datadict_fname.exists()

In [ ]:
df_redcap = utils.redcap.get_redcap_df(fname=redcap_fname, fname_datadict=redcap_datadict_fname)

In [ ]:
fname_pcirs = P_ROOT / "bx" / "PCIRS_6mo_FP_wide_scored_cononly_20240329.csv"
df_pcirs = pd.read_csv(fname_pcirs, index_col="ID", usecols=lambda col: col != "Unnamed: 0")
df_pcirs = df_pcirs.reset_index().rename(columns={"ID": "study_id"})
df_pcirs["study_id"] = df_pcirs["study_id"].apply(lambda id_: "sub-" + str(id_))
df_pcirs

In [ ]:
df_tmp = df_merged.loc[df_merged["session"] == "sixmonth"].drop(columns=["session", "gestational_age_weeks"]).pivot(columns=["region"]).droplevel(0, axis=1)
df_tmp.columns.name = ""
df_vol_pcirs = df_tmp.merge(df_pcirs, left_on="fsid", right_on="study_id")

In [ ]:
aseg_fname_newborn = P_ROOT / "aseg_newborn_brainvol.table.txt"
aseg_fname_sixmonth = P_ROOT / "aseg_sixmonth_brainvol.table.txt"

df_vol = utils.io.get_brainvol_df(aseg_fname_sixmonth)
df_vol = df_vol.pivot(columns=["region"]).droplevel(0, axis=1)
df_vol.columns.name = ""
df_vol

In [ ]:
df_vol_pcirs = df_pcirs.merge(df_vol, on="study_id")

In [ ]:
want_cols = [col for col in df_vol_pcirs.columns if "vol" in col.lower() or col == "senstot"]
df_reduced = df_vol_pcirs.loc[:, want_cols]
df_reduced

In [ ]:
sns.pairplot(df_reduced)

In [ ]:
df_reduced.corr()

In [ ]:
sns.lmplot(data=df_reduced, x="CerebralWhiteMatterVol", y="senstot")

In [ ]:
import statsmodels.formula.api as smf

mod = smf.ols(formula="senstot ~ CerebralWhiteMatterVol", data=df_reduced)
result = mod.fit()
result.summary()